In [ ]:
from bs4 import BeautifulSoup
import re
import requests
import pandas as pd 


In [ ]:
#Ingresar a la página principal de la NFL
page = requests.get("https://www.nfl.com/stats/player-stats/")
#Pasarlo a formato soup
soup = BeautifulSoup(page.content, 'html.parser')

In [ ]:
#Obtener los enlaces a cada pestaña del navbar
pestanhas=[x.a['href'] for x in soup.find("ul",{"class":"d3-o-tabs d3-o-tabs__nowrap"}).find_all('li')]
linksJugadores=[]
#Iterar para sacar los datos
for pestanha in pestanhas:
    statPage=requests.get("https://www.nfl.com"+pestanha)
    soup2 = BeautifulSoup(statPage.content, 'html.parser')
    table=soup2.find('table')
    #Obtener cada fila de la tabla de jugadores
    jugadores=table.find_all('tr')
    for jugador in jugadores:
        if(jugador.find('div',{'class':'d3-o-media-object__body'}) is None):continue
        else:
            #Agregar el link al perfil del jugador
            linksJugadores.append("https://www.nfl.com"+jugador.find('div',{'class':'d3-o-media-object__body'}).a['href'])